In [38]:
%load_ext autoreload
%autoreload 2
import time
import os
import seaborn as sns
import numpy as np
import pandas as pd
import sys

from mimas.tools.spectral_file.extract_ms1_feature import process_mzml_file, extract_ms2_spectra, find_features
import logging
from toolsets.spectra_operations import entropy_similarity_default
from toolsets.adduct_calculator import complete_adducts, complete_formula
# from toolsets.API_gets import complete_smiles
from toolsets.search import string_search, num_search
import toolsets.spectra_operations as so
from tqdm import tqdm
# from toolsets.features_by_alphapept import find_features_alphapept
# std_list_path= "/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
source_dir = "/Users/fanzhoukong/Documents/GitHub/Libgen_data/GT"
std_list = pd.read_csv(os.path.join(source_dir, "new_std_list.csv"))
enzyme_list = pd.read_csv(os.path.join(source_dir, "enzyme_list_cor"+".csv"))
# std_list.drop(['Count'], axis=1, inplace=True)
# std_list.rename({'For all (CAS or no CAS) Name to Smiles by pubchem':'smiles'}, axis=1, inplace=True)
# std_list.rename({'CAS to Inchi (CTS Proxy) for MoNA':'translated_inchikey'}, axis=1, inplace=True)
std_list = complete_formula(std_list)

std_list = std_list[~std_list['Formula'].isna()]
from toolsets.adduct_calculator import calculate_precursormz
std_list  = complete_adducts(std_list,adducts= ['[M+H]+','[M+NH4]+','[M+Na]+','[M-H2O+H]+'])
std_list.reset_index(inplace = True, drop = True)
std_list.to_csv(os.path.join(source_dir, "substrate_list.csv"), index = False)
new_std = {}
for col in std_list.columns:
    new_std[str(col)]= []

for index, row in std_list.iterrows():
    for col in std_list.columns:
        new_std[str(col)].append(row[str(col)])
    for gly in ['_glycoside', '_diglycoside']:
        for inner_col in ['Mix','Formula','CAS number','inchikey','smiles']:
            new_std[str(inner_col)].append(row[inner_col])
        new_std['Substrate Name'].append(row['Substrate Name']+gly)
        if gly == '_glycoside':
            for adduct in ['[M+H]+', '[M+NH4]+', '[M+Na]+','[M-H2O+H]+']:
                new_std[adduct].append(row[adduct]+162.0533)
        else:
            for adduct in ['[M+H]+', '[M+NH4]+', '[M+Na]+','[M-H2O+H]+']:
                new_std[adduct].append(row[adduct]+324.1066)
# new_std = pd.DataFrame.from_dict(new_std)
# new_std.to_csv(os.path.join(source_dir, "enlogated_std_list"+'.csv'), index = False)
new_std = pd.DataFrame.from_dict(new_std)

In [46]:
new_std

,Mix,Substrate Name,Formula,CAS number,inchikey,smiles,[M+H]+,[M+NH4]+,[M+Na]+,[M-H2O+H]+
0,1,Strophanthidinic Acid Lactone Acetate,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,445.2221,462.2486,467.2040,427.2115
1,1,Strophanthidinic Acid Lactone Acetate_glycoside,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,607.2754,624.3019,629.2573,589.2648
2,1,Strophanthidinic Acid Lactone Acetate_diglycoside,C25H32O7,No CAS#,RPIVIODUUUOQDD-UHFFFAOYSA-N,CC(=O)OC1CCC23C4CCC5(C(CCC5(C4CCC2(C1)O)OC3=O)...,769.3287,786.3552,791.3106,751.3181
3,1,Glutamine (L),C5H10N2O3,56-85-9,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,147.0764,164.1030,169.0584,129.0659
4,1,Glutamine (L)_glycoside,C5H10N2O3,56-85-9,ZDXPYRJPNDTMRX-VKHMYHEASA-N,C(CC(=O)N)[C@@H](C(=O)O)N,309.1297,326.1563,331.1117,291.1192
...,...,...,...,...,...,...,...,...,...,...
1774,15,Chlorogenic Acid_glycoside,C16H18O9,327-97-9,CWVRJTMFETXNAD-JUHZACGLSA-N,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,517.1557,534.1822,539.1376,499.1451
1775,15,Chlorogenic Acid_diglycoside,C16H18O9,327-97-9,CWVRJTMFETXNAD-JUHZACGLSA-N,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,679.2090,696.2355,701.1909,661.1984
1776,15,2'-deoxyadenosine,C10H13N5O3,13276-53-4,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,252.1091,269.1357,274.0911,234.0986
1777,15,2'-deoxyadenosine_glycoside,C10H13N5O3,13276-53-4,OLXZPDWKRNYJJZ-FSDSQADBSA-N,C1[C@H]([C@H](O[C@H]1N2C=NC3=C(N=CN=C32)N)CO)O,414.1624,431.1890,436.1444,396.1519


In [47]:
new_std.to_csv(os.path.join(source_dir, "enlongated_list.csv"), index= False)